# Preliminaries

## Installing prophet

Importing the base libraries like `io`, `os`, `sys`, `setuptools` and `tokenize` helps the dependencies resolution.

In [ ]:
%pip install prophet
%pip install plotly

## Import libraries

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import sys

from os import path
from plotly.subplots import make_subplots
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit


Importing the following library and calling the corresponding mount command is required only if you are operating in a Colab setting.

In [ ]:
root = path.join(".", "..")
content = path.join(root, "data")

## Prepare the dataset

We are ready to read the file, directly preparing it to be fed to our Prophet model (i.e. Prophet expects to find columns `ds` and `y` in the input DataFrame).

In [ ]:

pdata = path.join(content, "AirPassengers.csv")
nheader = ["ds", "y"]
datecols = ["ds"]

df = pd.read_csv(pdata,
                 # Tells pandas that the file contains a header (the first row,
                 # i.e. line 0) and to replace it with the one provided
                 # by 'names'
                 header = 0, names = nheader,
                 # Let also pandas know that one of the columns is a date
                 parse_dates = datecols)

Having exploited directly the 'read_csv' functionalities leave the code lighter and more readable.    
The same behaviour could have been achieved with the following calls    


```
df = pd.read_csv(pdata)
df['Month'] = pd.DatetimeIndex(df['Month'])
df = df.rename(columns =
  {
    'Month': 'ds',
    '#Passengers': 'y'
  })
```

which is a bit more verbose and error prone.    
    
Now let's quickly inspect our dataset...

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

... and visualize it

In [ ]:
fig = go.Figure(
    data = [go.Scatter(x = df.ds, y = df.y, mode = "lines")],
    layout = {
        "title": "Airline Passengers",
        "xaxis": {"title": "Date"},
        "yaxis": {"title": "Monthly Number of Airline Passengers"}
    }
)

fig.show()

Thanks to the peculiarity of this particular dataset, several properties can de directly derived from visual inspection

- **Trend**: thanks to the technological advance, increasing flights security and ticket's price drop, more and more people could afford an airplane trip. This is captured by the increasing slope in the data, also called *trend*.
- **Seasonality**: is identified as a pattern which repeats itself over and over in the data, with a fixed period (e.g. 12 for monthly data, 7 for weekly one). In the figure the monthly seasonal pattern is clearly visible, with a peak in the summer months when most of the people take their vacations.
-- *Additive seasonality*: we can also distinguish two modalities for a seasonal pattern, an *additive* one (i.e. the magnitude of the seasonality does not vary with the level of the data) or a *multiplicative* modality.

For the dataset under inspection and for what we just said, we can conclude that the seasonality in the Airline Passengers case is multiplicative. To better explain this latter concept let's decompose the data in the old fashion, with moving averages

In [ ]:

# The data have an even period (i.e. 12 months), hence the trend is estimated
# as a centered moving average (i.e. 2xm moving average)
df["atrend"] = df.y.rolling(12, center=True).mean()

# Remove the trend from the original. In the additive case we subtract the
# trend estimation. In the multiplicative hypothesis we divide by the
# estimation.
df["adty"] = df.y - df.atrend

# The season for each month is simply the average of the values for that month
# over the whole data range.
df["aseason"] = df.groupby(df.ds.dt.month).adty.transform("mean")

decomposition_fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    subplot_titles=["Trend", "Season", "Reconstruction"])
decomposition_fig.update_layout(showlegend=False)
decomposition_fig.add_trace(
    go.Scatter(x = df.ds, y = df.atrend),
    row=1, col=1
)
decomposition_fig.add_trace(
    go.Scatter(x = df.ds, y = df.aseason),
    row=2, col=1
)
decomposition_fig.add_trace(
    go.Scatter(x = df.ds, y = df.y,
               mode="lines", line={"color": "green"}),
    row=3, col=1
)
decomposition_fig.add_trace(
    go.Scatter(x = df.ds, y = df.atrend + df.aseason,
               mode="markers",
               marker={
                   "color": "rgba(0, 0, 0, 0)",
                   "line": {"width": 1,"color": "green"}
               }),
    row=3, col=1
)

In [ ]:
df["trend"] = df.y.rolling(12, center=True).mean()
df["dty"] = df.y / df.trend
df["season"] = df.groupby(df.ds.dt.month).dty.transform("mean")

decomposition_fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    subplot_titles=["Trend", "Season", "Reconstruction"])
decomposition_fig.update_layout(showlegend=False)
decomposition_fig.add_trace(
    go.Scatter(x = df.ds, y = df.trend),
    row=1, col=1
)
decomposition_fig.add_trace(
    go.Scatter(x = df.ds, y = df.season),
    row=2, col=1
)
decomposition_fig.add_trace(
    go.Scatter(x = df.ds, y = df.y,
               mode="lines", line={"color": "green"}),
    row=3, col=1
)
decomposition_fig.add_trace(
    go.Scatter(x = df.ds, y = df.trend * df.season,
               mode="markers",
               marker={
                   "color": "rgba(0, 0, 0, 0)",
                   "line": {"width": 1,"color": "green"}
               }),
    row=3, col=1
)

The difference is subtle, but the Devil is in the details.

#Prophet forecasting


##Air Passengers Data

As we said the Airline Passengers dataset exposes a trend and monthly `multiplicative` seasonality. This is exactly what we are going to ask to Prophet. For the moment we will keep the trend `linear` (Prophet default) and ask only for the yearly seasonality.    
    
Prophet does not handle well monthly-data and sub-daily data and could lead to non-sensical results. However this let us introduce the concepts of regressors.    
The goal here is to assign a certain "weight" to each month of the year, the weight will resemble how much that particular month contributes to the forecast. To achieve this we introduce 12 dummy variables, one for each month of the year, that will be passed as regressors to the model.

In [ ]:
months_dummy = ['is_jan', 'is_feb', 'is_mar', 'is_apr', 'is_may', 'is_jun',
                'is_jul', 'is_aug', 'is_sep', 'is_oct', 'is_nov', 'is_dec']
for i, month in enumerate(months_dummy):
  df[month] = (df['ds'].dt.month == i + 1).values.astype('float')

df.head()

In [ ]:
linear = Prophet(
    daily_seasonality=False, # we are not interested in this seasonality
    yearly_seasonality=False, # we are not interested in this seasonality
    weekly_seasonality=False, # we are not interested in this seasonality
    seasonality_mode="multiplicative", # ask for a multiplicative treatment
    mcmc_samples=200) # useful to estimate uncertainty on components

# Add the 12 regressors
for month in months_dummy:
  linear.add_regressor(month)

linear.fit(df, show_progress=False)

In [ ]:
future_dates = linear.make_future_dataframe(periods=36, freq='MS')

for i, month in enumerate(months_dummy):
    future_dates[month] = (future_dates['ds'].dt.month == i + 1).values.astype('float')

future_dates.head()

In [ ]:
forecast = linear.predict(future_dates)
forecast.head()

In [ ]:
fig = linear.plot(forecast);

In [ ]:
linear.plot_components(forecast);

You could have possibly achieved the same results calling

In [ ]:
ylinear = Prophet(
      daily_seasonality=False,
      yearly_seasonality="auto", # (default) just to highlith the difference
      weekly_seasonality=False,
      seasonality_mode="multiplicative",
      mcmc_samples=200
    ).fit(df, show_progress=False);
yforecast = ylinear.predict(future_dates);

In [ ]:
yfig = ylinear.plot(yforecast);

In [ ]:
ylinear.plot_components(yforecast);

But, as you can see, even if the final forecast seems correct, the seasonal component makes no sense because the model searches for a mid-month information which is not there.

##Metrics **(TO BE REVIEWED)**

The default `cross_validation` function available in Prophet produce daily date range by default. Internally `Timedelta`s are used to generate the `cutoff`s instead of the `daterange` employed in the `make_future_dataframe()` function. `Timedelta` indeed expects a different set of frequencies, raging from weeks to nanoseconds.     
This is not suitable for the dataset we are working with: the number of observed passengers are aggregated on the first day of each month, we would like to forecast the number of passengers for the next `horizon` months - with `period` - and keep the first day of the month as a reference for the time index.    
A combination of `horizon` and `period` expressed in weeks or days will generate a misalignment.    
    
Keeping the same structure of the original `cross_validation` we use a custom version of it.

In [ ]:
custom_package = path.join(root, "custom_prophet")
if not custom_package in sys.path: sys.path.append(custom_package) # Hack way to import a local module

from custom_diagnostics import cross_validation
from prophet.diagnostics import performance_metrics


In [ ]:
df_cv = cross_validation(linear, 12, freq = pd.infer_freq(df['ds']),
                         initial=pd.Timestamp("1952-12-01"))

In [ ]:
df_cv.head()

Setting `initial = "1952-12-01"` and `horizon = 12` we are: using the first 4 years before the first `cutoff` date; asking for a whole year prediction after each `cutoff` date. Not having specified a `period`, it is automatically set to `period = 0.5 * horizon = 6` months.    
    
8 years are left as validation period - from 1953 to 1960 included - leading to *15* **overlapping** *cutoffs*: *2* for each year between 1953 and 1959, *1* for the last year since forecasting from June 1960 to June 1961 will fall outside the max historical date available. We can visual the prediction at different cutoff dates grouping the cross validation Dataframe by the cutoff date and plotting each group.

In [ ]:
grouped = df_cv.groupby(by=df_cv['cutoff'], sort=True)
print("Number of groups: %d" % len(grouped.groups))
grouped_data = [\
    go.Scatter(x = group.ds, y = group.yhat, mode = "lines")\
    for (_, group) in grouped\
  ]

cv_fig = go.Figure(
    data = grouped_data,
    layout = {
        "title": "Cross Validation",
        "xaxis": {"title": "Date"},
        "yaxis": {"title": "Monthly Number of Airline Passengers"}
    }
)

cv_fig.show()

Keeping `initial` and `horizon` as before but providing a `period = 12`, we have *8* **non-overlapping** *cutoffs*, 1 for each year from 1953 to 1960. Again, grouping the cross validation Dataframe by the cutoff date and plotting each group we can visual the prediction.

In [ ]:
df_cv_wperiod = cross_validation(linear, 12, freq = pd.infer_freq(df['ds']),
                         period = 12,
                         initial=pd.Timestamp("1952-12-01"));

df_cv_wperiod.head()

In [ ]:
grouped_wperiod = df_cv_wperiod.groupby(by=df_cv_wperiod['cutoff'], sort=True)
print("Number of groups: %d" % len(grouped_wperiod.groups))
grouped_data = [\
    go.Scatter(x = group.ds, y = group.yhat, mode = "lines")\
    for (_, group) in grouped_wperiod\
  ]

cv_wperiod_fig = go.Figure(
    data = grouped_data,
    layout = {
        "title": "Cross Validation",
        "xaxis": {"title": "Date"},
        "yaxis": {"title": "Monthly Number of Airline Passengers"}
    }
)

cv_wperiod_fig.show()

Time to measure the performance of our model. Surprisingly the function `performance_metrics` expose a `monthly` flag which can be set to handle monthly data. Internally the function is taking the cross validation dates column `ds` and resampling it with a monthly frequency.    
In our case the latter operation will be useless but lets us re-use the legacy function without having to implement our own version, as we did instead for `cross_validation`.

In [ ]:
df_p = performance_metrics(df_cv, monthly = True)
df_p

In [ ]:
df_p.mean()